<img src="https://brand.umich.edu/assets/brand/style-guide/logo-guidelines/U-M_Logo-Horizontal-Hex.png" alt="Drawing" style="width: 300px;" align="left"/><br>
    
## Week 4: Building a learning analytics dashboard

In this week, you will be working to build an interactive dashboard to visualize prediction results and other important information about the course and the students. Keep in mind that the dashboard is aimed at non-technical audience, such as school administrator and/or instructors. You will be using the [jupyter-dash extension](https://github.com/plotly/jupyter-dash) to build a [Plotly Dash](https://github.com/plotly/dash) app interactively within Jupyter environments.

Dash is the most downloaded, trusted Python framework for building ML & data science web apps. You can check out the documentation and some tutorials for Plotly Dash [here](https://dash.plotly.com/).

Resources:
* [A medium blog - Introducing JupyterDash](https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e)
* [Plotly Dash documentation](https://dash.plotly.com/)
* [A Youtube tutorial on dash](https://www.youtube.com/watch?v=hSPmj7mK6ng)
* [Dash gallery](https://dash-gallery.plotly.host/Portal/)

The dashboard should enable users to quickly extract insights to answer the following questions:
* Who are the students in my course? (20 pts)
* Which students are likely to drop out or fail my course? (20 pts)
* How do my course compare to other courses in terms of pass_rate and dropout_rate? (20 pts)
* What is the grade distribution of my course? (20 pts)
* Integrate everything into a single dashboard (20 pts)

The dashboard will be manually graded based on the following rubric: TBD

You can also check out [OU Analyse](https://analyse.kmi.open.ac.uk/), screenshot below, as an example of how the authors of the dataset have developed a dashboard for their institution. You can request a demo of the dashboard by enter your email.
![OU Analyse](https://analyse.kmi.open.ac.uk/resources/images/project_info/screenshot_01.png)

# Loading libraries

In [14]:
import pandas as pd
import numpy as np

# Import jupyter dash
from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
import dash_core_components as dcc
import dash_html_components as html

# Import plotly
import plotly.graph_objs as go
import plotly.express as px

# Set up jupyter proxy
JupyterDash.infer_jupyter_proxy_config()

# A quick tutorial

In essence, a plotly-dash dashboard consists of 3 components:
* The **dash components** (e.g., dropdown, slider, checklist, etc.). See the documentation [here](https://dash.plotly.com/dash-core-components)
* The **plotly** graphs (e.g., linegraph, scatter plot, heatmap, etc.). See the documentation [here](https://plotly.com/python/)
* The **callback** to connects the dash components to plotly graphs, making it an interactive dashboard. See the documentation [here](https://dash.plotly.com/basic-callbacks) 



In [15]:
# Let's import a sample dataframe
df = pd.read_csv('assets/course_passrate.csv')
df

,code_module,code_presentation,pass_rate,dropout_rate
0,AAA,2013J,0.860681,0.156658
1,AAA,2014J,0.846154,0.180822
2,BBB,2013B,0.636292,0.285795
3,BBB,2013J,0.672944,0.287886
4,BBB,2014B,0.647373,0.303782
5,BBB,2014J,0.746598,0.326789
6,CCC,2014B,0.638728,0.463843
7,CCC,2014J,0.714286,0.431145
8,DDD,2013B,0.585534,0.331543
9,DDD,2013J,0.659507,0.351393


## Step 1. The Dash components (i.e. layouts)

In [16]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = df['code_module'].unique()

# Step 1

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is the title"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='AAA' # the default is code_module AAA
            )])

# Run the app
app.run_server(mode="inline")

# You will see we have the dropdown menu but nothing happens yet

## Step 2 & 3. Callback and create plotly graph

In [17]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = df['code_module'].unique()

# Step 1

app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a bar chart"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph-with-dropdown'),
    
            # Create a dropdown menu based on code_module
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators1],
                value='AAA' # the default is code_module AAA
            )])

# Step 2
# Callback using input from dropdown menu to generate graph
# You can have multiple inputs and multiple outputs
@app.callback(
    dash.dependencies.Output('graph-with-dropdown', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(code_module):
    filtered_df = df[(df.code_module == code_module)]
    figure = px.bar(filtered_df, x='code_presentation', y='pass_rate')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode="inline", port =8060)

## Let's plot a scatter plot for pass_rate and dropout_rate

In [18]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_presentation
available_indicators1 = df['code_presentation'].unique()

# Step 1
app.layout = html.Div([
            # Create a html title for the dashboard
            html.H1("This is a scatter plot"),
    
            # Create a graph, we will configure the graph using plotly express in step 3 
            dcc.Graph(id='graph'),
    
            # Create a checklist based on code_presentation
            dcc.Checklist(
                    id = 'checklist',
                    options=[{'label': i, 'value': i} for i in available_indicators1],
                    value=available_indicators1 # Default values contain all code_presentation
            )
])

# Step 2
# Callback using inputs from the checklist to generate the graph
@app.callback(
    dash.dependencies.Output('graph', 'figure'),
    [dash.dependencies.Input('checklist', 'value')])

# Step 3
# Define the graph with plotly express
def update_figure(code_presentation):
    filtered_df = df[(df.code_presentation.isin(code_presentation))]
    figure = px.scatter(filtered_df, 
                     x="pass_rate", 
                     y="dropout_rate", 
                     color="code_presentation",
                     hover_name='code_module')
    return figure # You must return all the output(s) in step 2
    
# Run the app
app.run_server(mode="inline")

**Build the demo app**

In [19]:
df = pd.read_csv('assets/country_indicators.csv')
available_indicators = df['Indicator Name'].unique()
df

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN
...,...,...,...,...
36955,Zimbabwe,"Industry, value added (% of GDP)",2007,33.074953
36956,Zimbabwe,"Inflation, GDP deflator (annual %)",2007,0.894887
36957,Zimbabwe,"Life expectancy at birth, total (years)",2007,44.177756
36958,Zimbabwe,Population density (people per sq. km of land ...,2007,34.374559


In [22]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Design app layout
app.layout = html.Div([
    html.Div([

        html.Div([
            # Create a dropdown menu on x-axis
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            
            # Create a multiple choice menu on x-axis
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            # Create a dropdown menu on y-axis
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            # Create a multiple choice menu on y-axis
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),
    
    # Create a dashboard which consists of a scatter plot, x-time-series, and y-time-series
    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),
    
    # Create a slider on year
    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

# Call back function
# It will take the 5 inputs and produce 1 output which is the scatter plot figure
@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])

# Define update_graph function
# It basically filters the df based on users' input (e.g. dropdown menu, slider, multiple choice)
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [dict(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }


def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

# Call back for time-series graph on x-axis
@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)

# Call back for time-series graph on y-axis
@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

In [23]:
# Run the app inside jupyter notebook
app.run_server(mode="inline", port =8051)


---

## Part A (20 points) 

Create a dashboard that shows the a list of students and their associated prediction for pass/fail the course. The dashboard should allow instructors to filter by code_module and code_presentation     
You can use the predicted probabilities and predicted label for students in 'assets/student_pred.csv'

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server


# Step 1: Create a layout (title, dropdown menu, slider, etc...)

# Step 2: Callback to connect input(s) to output(s)

# Step 3: Define the graph with plotly express

    
# Run the app
app.run_server(mode="inline", port =8052)

---

## Part B (20 points)

Create a dashboard that shows the submission rate of each of the first 3 TMAs and the grade distribution for each TMA. The dashboard should allow instructors to filter by code_module and code_presentation  

You can use the pass_rate and submission_rate in 'assets/course_passrate.csv'

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server


# Step 1: Create a layout (title, dropdown menu, slider, etc...)

# Step 2: Callback to connect input(s) to output(s)

# Step 3: Define the graph with plotly express


    
# Run the app
app.run_server(mode="inline", port =8053)

---

## Part C (20 points)

Create a dashboard that shows the weekly sum click on VLE. It show allows instructors to filter by code_module, code_presentation, VLE_activity_type (e.g., homepage, forums, etc.)

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server


# Step 1: Create a layout (title, dropdown menu, slider, etc...)

# Step 2: Callback to connect input(s) to output(s)

# Step 3: Define the graph with plotly express

    
# Run the app
app.run_server(mode="inline", port =8054)

---

## Part D (20 points)

Create a dashboard that shows for each student: 
* Their prediction
* Their grade of the first TMA assignments
* Their weekly VLE activities

It should allow instructors to filter by id_student

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server


# Step 1: Create a layout (title, dropdown menu, slider, etc...)

# Step 2: Callback to connect input(s) to output(s)

# Step 3: Define the graph with plotly express


# Run the app
app.run_server(mode="inline", port =8055)

---

## Part E (20 points)

Intergrate everything into a single dashboard. It must allows for cross filtering (e.g, all the figures are updated as user select an input). Imagine an instructor will look at part A to pick out students who are at-risk, and then drill down to individual students in part D.

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server


# Step 1: Create a layout (title, dropdown menu, slider, etc...)

# Step 2: Callback to connect input(s) to output(s)

# Step 3: Define the graph with plotly express


# Run the app
app.run_server(mode="inline", port =8056)